In [ ]:
## For colab runs
# from google.colab import drive
# drive.mount('/content/gdrive')

# !mkdir -p data
# !cp /content/gdrive/My\ Drive/uploads/a9a.2 ./data # load this data to the specified dir on your drive first

# DGM Minimal Environment 

TODO: Add to `dummy_consensus_variation` check of a graph connectedness

In [ ]:
import torch
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_svmlight_file
from src.objectives import (
    LeastSquares, LogRegression,
    StochLeastSquares, StochLogRegression)
from src.methods import EXTRON, DIGONing, DAGDON, DOGSGD
from src.utils import (
    consensus_variation,
    PythonGraph)

# from src.sparse import objectives as obj

In [ ]:
num_nodes = 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
A, b = load_svmlight_file('data/a9a')
A = torch.Tensor(A.todense()).to(device)
b = torch.Tensor(b).to(device)

In [ ]:
# For simulating a graph evolution,
# only graphs like 'erdos_renyi' is appropriate

p = .3
graph = 'random_geometric'
# graph = 'erdos_renyi'
# graph = 'path'
# graph = 'cycle'
# graph = 'complete'

F = LogRegression(A, b, num_nodes)
Fs = StochLogRegression(A, b, num_nodes,)
X0 = torch.zeros(num_nodes, A.size(1)).to(device)
gen = PythonGraph(F, graph, p).gen
# gen = lambda: dummy_consensus_variation(F, graph, p)
# gen = lambda: (None, consensus_variation(F, p))

opt1 = EXTRON(F, gen, alpha=4.)
opt2 = DIGONing(F, gen, alpha=4.)
opt3 = DAGDON(F, gen, L=.2, mu=0., T=20)

gen_ = lambda : gen()[1]
opt4 = DOGSGD(Fs, gen_, alpha=4.)

In [ ]:
%%time
n_iters = 100
opt1.run(X0, n_iters=n_iters);
opt2.run(X0, n_iters=n_iters);
opt3.run(X0, n_iters=n_iters);
opt4.run(X0, n_iters=n_iters);

In [ ]:
plt.figure(figsize=(8, 8))
plt.plot(opt1.logs['fn'], label='EXTRA');
plt.plot(opt2.logs['fn'], '--', label='DIGing');
plt.plot(opt3.logs['fn'], label='AGD');
plt.plot(opt4.logs['fn'], label='DGSGD')
plt.title('Optimization Functional Value over Iteration Number', size=20)
plt.xlabel('# k', size=20)
plt.ylabel('f(x)', size=20);
plt.legend();

In [ ]:
plt.figure(figsize=(8, 8))
plt.plot(opt1.logs['dist2con'], label='EXTRA');
plt.plot(opt2.logs['dist2con'], '--', label='DIGing');
plt.plot(opt3.logs['dist2con'], label='AGD');
plt.plot(opt4.logs['dist2con'], label='DGSGD');
plt.title(r'$||X(I-11^T)||^2$', size=20)
plt.xlabel('# k', size=20)
plt.legend();